In [39]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
from api_info import *
from credentials import *
from api_functions import *
from stat_functions import *

In [14]:
d = retrieve_espn_data(swid,espn_s2,league_id)
data = clean_data(d)

Player has irregular eligible slots: Elias N. Pettersson has slots [10, 4, 6, 7, 8]


In [15]:
skater_cats = [
    "FOW",
    "assists",
    "average_time_on_ice",
    "blocks",
    "goals",
    "hits",
    "penalty_minutes",
    "plus_minus",
    "power play points",
    "shots_on_goal",
]

In [7]:
possible_max_categories = skater_cats+['short handed goals','short handed assists','game winning goals','special team points','clean_sheets','shots_at_them','saves','over_time_loss','FOPercent']
possible_min_categories = ['goals against average','goals against']

award_df = data[~data["LeagueTeamID"].isna()].copy()

In [16]:
for cat in possible_max_categories:
    try:
        award_df['test'] = award_df[cat+'AT']-award_df[cat+'proj']
        top = award_df[["Name","LeagueTeamID",cat+'AT',cat+'proj',"games_playedAT",'test']].sort_values(by='test',ascending=False)
        if (7 in list(top.iloc[:4]['LeagueTeamID'])) :
            print()
            print(cat)
            print(top.iloc[:4])
    except KeyError:
        print(f"No columns: {cat}")


blocks
                  Name  LeagueTeamID  blocksAT  blocksproj  games_playedAT  \
461      Tony DeAngelo           3.0  2.111111    0.674419             9.0   
107          Alex Tuch           6.0  1.490909    0.671053            55.0   
409        Jake McCabe           6.0  2.062500    1.364865            48.0   
247  Alexander Romanov           7.0  2.595238    2.038462            42.0   

         test  
461  1.436693  
107  0.819856  
409  0.697635  
247  0.556777  

penalty_minutes
                 Name  LeagueTeamID  penalty_minutesAT  penalty_minutesproj  \
287      Drew Doughty           1.0           1.142857             0.590164   
123        Jamie Benn           5.0           1.052632             0.536585   
87         Timo Meier           7.0           0.896552             0.444444   
121  Rasmus Andersson           3.0           0.857143             0.405063   

     games_playedAT      test  
287             7.0  0.552693  
123            57.0  0.516046  
87          

In [19]:
award_df.sort_values(by="hatricksAT",ascending=False)[["Name","LeagueTeamID","hatricksAT","games_playedAT"]]

,Name,LeagueTeamID,hatricksAT,games_playedAT
6,Mikko Rantanen,3.0,0.053571,56.0
29,Alex Ovechkin,6.0,0.048780,41.0
41,John Tavares,8.0,0.040000,50.0
140,Mark Scheifele,8.0,0.035088,57.0
246,Patrik Laine,2.0,0.034483,29.0
...,...,...,...,...
1227,Kaapo Kakko,5.0,NaN,56.0
1267,Jackson LaCombe,7.0,NaN,49.0
1360,Casey DeSmith,7.0,NaN,NaN
1467,Nikita Zadorov,5.0,NaN,58.0


In [21]:
birth_dates_df = pd.read_csv("player_bios.csv",header=0)
birth_dates_df["Age"]=pd.to_datetime('today').year-pd.to_datetime(birth_dates_df["DOB"]).dt.year
birth_dates_df

,Player,Team,S/C,Pos,DOB,Birth City,S/P,Ctry,Ntnlty,Ht,...,Draft Yr,Round,Overall,1st Season,HOF,GP,G,A,P,Age
0,Jesse Puljujarvi,PIT,R,R,1998-05-07,Alvkarleby,--,SWE,FIN,76,...,2016,1,4,20162017,N,12,1,5,6,27
1,Taylor Raddysh,WSH,R,R,1998-02-18,Caledon,ON,CAN,CAN,75,...,2016,2,58,20212022,N,12,2,3,5,27
2,Pierre-Luc Dubois,WSH,L,L,1998-06-24,Ste-Agathe-des-Monts,QC,CAN,CAN,76,...,2016,1,3,20172018,N,12,1,6,7,27
3,Nathan Bastian,NJD,R,R,1997-12-06,Kitchener,ON,CAN,CAN,76,...,2016,2,41,20182019,N,12,2,3,5,28
4,Filip Hronek,VAN,R,D,1997-11-02,Hradec Kralove,--,CZE,CZE,72,...,2016,2,53,20182019,N,12,0,4,4,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Justin Faulk,STL,R,D,1992-03-20,South St. Paul,MN,USA,USA,72,...,2010,2,37,20112012,N,14,1,4,5,33
701,Brock Nelson,NYI,L,C,1991-10-15,Warroad,MN,USA,USA,76,...,2010,1,30,20132014,N,14,5,2,7,34
702,Kevin Hayes,PIT,L,R,1992-05-08,Dorchester,MA,USA,USA,77,...,2010,1,24,20142015,N,14,3,1,4,33
703,Jaden Schwartz,SEA,L,C,1992-06-25,Wilcox,SK,CAN,CAN,70,...,2010,1,14,20112012,N,14,3,5,8,33


In [22]:
for i in range(1,9):
    award_team_df= award_df[award_df["LeagueTeamID"]==i]
    ages = birth_dates_df[birth_dates_df["Player"].isin(list(award_team_df["Name"]))]["Age"]
    print(i,ages.sum()/len(ages))

1 29.375
2 30.17391304347826
3 28.64
4 29.46153846153846
5 28.333333333333332
6 30.52173913043478
7 28.75
8 29.652173913043477


In [29]:
league_team_awards ={k:{} for k in range(1,9)}

for cat in possible_max_categories:
    if cat not in ["games_played"] and cat+"AT" in award_df.columns:
        
        cat_term = cat+"AT"
        if cat =="FOPercent":
            cat_df =award_df[award_df["FOWAT"]>10][["Name","LeagueTeamID",cat_term,"games_playedAT"]].copy()
        elif cat=="average_time_on_ice":
            cat_df=award_df[award_df["Position"]!="GK"][["Name","LeagueTeamID",cat_term,"games_playedAT"]].copy()
        else:
            cat_df = award_df[["Name","LeagueTeamID",cat_term,"games_playedAT"]].copy()
        cat_df[cat_term]=cat_df[cat_term]*cat_df["games_playedAT"]
        top = cat_df.sort_values(by=cat_term,ascending=False).iloc[0:4]
        if top[cat_term].iloc[0]==top[cat_term].iloc[1]:
            print()
            print("DRAW")
            print(cat_term)
            print(top)
        else:
            league_team_awards[int(top.iloc[0]["LeagueTeamID"])][cat]=top.iloc[0]


DRAW
short handed assistsAT
                  Name  LeagueTeamID  short handed assistsAT  games_playedAT
131      Brandon Hagel           1.0                     4.0            56.0
96      Jake Sanderson           8.0                     4.0            56.0
53   Aleksander Barkov           2.0                     3.0            48.0
122      Zach Werenski           8.0                     2.0            56.0


In [30]:
league_team_awards

{1: {'goals': Name              Leon Draisaitl
  LeagueTeamID                 1.0
  goalsAT                     42.0
  games_playedAT              57.0
  Name: 8, dtype: object,
  'short handed goals': Name                    Sam Reinhart
  LeagueTeamID                     1.0
  short handed goalsAT             5.0
  games_playedAT                  58.0
  Name: 35, dtype: object,
  'game winning goals': Name                    Leon Draisaitl
  LeagueTeamID                       1.0
  game winning goalsAT               9.0
  games_playedAT                    57.0
  Name: 8, dtype: object,
  'FOPercent': Name              Vincent Trocheck
  LeagueTeamID                   1.0
  FOPercentAT               34.27281
  games_playedAT                57.0
  Name: 43, dtype: object},
 2: {},
 3: {'assists': Name              Nathan MacKinnon
  LeagueTeamID                   3.0
  assistsAT                     66.0
  games_playedAT                59.0
  Name: 0, dtype: object,
  'shots_on_goal': N

In [ ]:
birth_places = pd.read_csv("birthplaces_BC.csv",header=1)
birth_places

,Rk,Player,From,To,Pos,GP,G,A,PTS,+/-,...,GP.1,W,L,T/O,SV%,GAA,Birth City,Birth Date,Death Date,-9999
0,1,Bryan Adams,2000,2001,C,11,0,1,1,-5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Fort St. James,1977-03-20,NaN,adamsbr01
1,2,Greg Adams,1981,1990,LW,544,84,143,227,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Duncan,1960-05-31,NaN,adamsgr01
2,3,Greg Adams,1985,2001,LW,1056,355,388,743,-48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Nelson,1963-08-15,NaN,adamsgr02
3,4,Bruce Affleck,1975,1984,D,280,14,66,80,-82.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Salmon Arm,1954-05-05,NaN,afflebr01
4,5,Micah Aivazoff,1994,1996,C,92,4,6,10,-9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Powell River,1969-05-04,NaN,aivazmi01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5458,527,Ed Zeniuk,1955,1955,D,2,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Landis,1933-03-08,1996-04-14,zeniued01
5459,528,Marty Zoborosky,1945,1945,D,1,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Moose Jaw,1916-11-12,1989-06-01,zoborma01
5460,1,Bryon Baltimore,1980,1980,D,2,0,0,0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Whitehorse,1952-08-26,NaN,baltibr01
5461,2,Dylan Cozens,2021,2025,C,294,68,103,171,-43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Whitehorse,2001-02-09,NaN,cozendy01


In [33]:
for i in range(1,9):
    award_team_df = award_df[award_df["LeagueTeamID"]==i]
    print(i,len(award_team_df[award_team_df["Name"].isin(birth_places["Player"])]))

1 12
2 7
3 9
4 14
5 11
6 11
7 7
8 15


In [36]:
award_df['test']=award_df['shots_on_goalAT']+award_df['hitsAT']+award_df['blocksAT']
award_df.sort_values(by='test',ascending=False)[['Name','test']]

,Name,test
7,Brady Tkachuk,8.071429
407,Kiefer Sherwood,8.018868
71,MacKenzie Weegar,6.964286
74,Moritz Seider,6.947368
255,Radko Gudas,6.854545
...,...,...
455,Mackenzie Blackwood,NaN
476,Anthony Stolarz,NaN
583,Kevin Lankinen,NaN
906,Jeremy Lauzon,NaN
